In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import datetime as dt


In [2]:

# Load Dataset
df = pd.read_csv("netflix_single_user_balanced_pattern_v3.csv")

# Add Temporal Features
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)


In [3]:

# Encode Categorical Features
label_encoder_group = LabelEncoder()
label_encoder_name = LabelEncoder()
label_encoder_device = LabelEncoder()

df['service_group_encoded'] = label_encoder_group.fit_transform(df['service_group'])
df['service_name_encoded'] = label_encoder_name.fit_transform(df['service_name'])
df['device_type_encoded'] = label_encoder_device.fit_transform(df['device_type'])


In [4]:

# Select Features and Target
features = [
    'hour', 'day_of_week', 'is_weekend', 'usage_minutes', 'usage_percentage',
    'signal_strength', 'packet_loss_rate', 'latency', 'jitter_ms', 'traffic_spike',
    'bandwidth_speed_per_sec_mbps', 'buffer_occupancy', 'device_type_encoded'
]
target_group = 'service_group_encoded'


In [5]:

# Scale Features
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])


In [ ]:

# Prepare Data for LSTM
sequence_length = 96
X = []
y_group = []

for i in range(len(df) - sequence_length):
    X.append(df[features].iloc[i:i + sequence_length].values)
    y_group.append(df[target_group].iloc[i + sequence_length])

X = np.array(X)
y_group = np.array(y_group)


In [ ]:

# Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_group_train, y_group_test = train_test_split(X, y_group, test_size=0.2, random_state=42)


In [ ]:

# Build LSTM Model
input_shape = X_train.shape[1:]

model_group = Sequential()
model_group.add(LSTM(128, input_shape=input_shape, return_sequences=True))
model_group.add(Dropout(0.2))
model_group.add(LSTM(64))
model_group.add(Dropout(0.2))
model_group.add(Dense(64, activation='relu'))
model_group.add(Dropout(0.2))
model_group.add(Dense(len(label_encoder_group.classes_), activation='softmax'))


In [ ]:

# Compile Model
model_group.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Train Model
model_group.fit(X_train, y_group_train, validation_data=(X_test, y_group_test), epochs=5, batch_size=64)


In [ ]:

# Predict Services for Specific Time Input
def predict_services(hour, minute, day_of_week):
    """
    Predict the service group and name for a specific time/hour/minute input.
    :param hour: Hour of the day (0-23).
    :param minute: Minute of the hour (0-59).
    :param day_of_week: Day of the week (0=Monday, 6=Sunday).
    :return: Predicted service group and service name.
    """
    # Create a feature row with given inputs
    test_input = {
        'hour': hour / 23,  # Scale to 0-1
        'day_of_week': day_of_week / 6,  # Scale to 0-1
        'is_weekend': 1 if day_of_week >= 5 else 0,
        'usage_minutes': 0.5,  # Example default (scaled)
        'usage_percentage': 0.5,  # Example default (scaled)
        'signal_strength': 0.5,  # Example default (scaled)
        'packet_loss_rate': 0.01,  # Example default (scaled)
        'latency': 0.5,  # Example default (scaled)
        'jitter_ms': 0.1,  # Example default (scaled)
        'traffic_spike': 0,  # Example default
        'bandwidth_speed_per_sec_mbps': 0.5,  # Example default (scaled)
        'buffer_occupancy': 0.5,  # Example default (scaled)
        'device_type_encoded': 0.5  # Example default (scaled)
    }
  

In [ ]:
  
    # Convert to DataFrame and array
    test_df = pd.DataFrame([test_input])
    test_array = test_df.values.reshape(1, 1, -1)  # LSTM expects 3D input

    # Predict service group
    predicted_group = model_group.predict(test_array)
    predicted_group_class = np.argmax(predicted_group, axis=1)
    predicted_service_group = label_encoder_group.inverse_transform(predicted_group_class)

    return predicted_service_group[0]


In [ ]:

# Example Prediction
hour = 20  # 8:00 PM
minute = 30
day_of_week = 4  # Friday

predicted_service_group = predict_services(hour, minute, day_of_week)
print(f"Predicted Service Group for {hour}:{minute} on day {day_of_week}: {predicted_service_group}")
